<a href="https://colab.research.google.com/github/Dhilip546/intern/blob/main/intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156236 sha256=146f8f834e89badb07120a159b1d01cd7886390a05745b6db361438af4d2b6fd
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


What are recommandation Engine?


  A recommendation engine, also known as a recommender system or recommendation system, is a class of machine learning algorithms and information retrieval techniques used in various applications to provide personalized recommendations to users. The primary goal of recommendation engines is to suggest relevant items or content to users based on their preferences, past behavior, or characteristics. These systems are widely used in e-commerce platforms, content streaming services, social networks, and more.

> Indented block



In [ ]:
#Importing necessary libraries
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from datetime import datetime
from surprise.model_selection import GridSearchCV
from surprise.model_selection import KFold
from sklearn.metrics import mean_squared_error, recall_score, precision_score, ndcg_score
import numpy as np


In [ ]:
#Data collection
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Online Retail.csv')


Mounted at /content/drive


In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [ ]:
 df['CustomerID'].unique()

array([17850., 13047., 12583., ..., 13298., 14569., 12713.])

In [ ]:
 df['CustomerID'].count()

406829

In [ ]:
 df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
#Data Filtering
reader = Reader(rating_scale=(0, 5))
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
data = Dataset.load_from_df(df[['CustomerID', 'Description', 'Quantity']], reader)

In [ ]:
kf = KFold(n_splits=5, random_state=42)

In [ ]:
rmse_scores = []
recall_scores = []
precision_scores = []
mrr_scores = []
map_at_k_scores = []
ndcg_scores = []

In [ ]:
#Model Building
for trainset, testset in kf.split(data):
    model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02, biased=True)
    model.fit(trainset)
    predictions = model.test(testset)

    # To Extract ground truth ratings and predicted ratings
    ground_truth = [rating.r_ui for rating in predictions]
    predicted_ratings = [rating.est for rating in predictions]

    # To Calculate RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(ground_truth, predicted_ratings))
    rmse_scores.append(rmse)

    # Setting a threshold value
    threshold = 4.0

    true_positives = []
    recommended_items = []
    for prediction in predictions:
        if prediction.r_ui >= threshold:  # True
            true_positives.append(1)
        else:  # False
            true_positives.append(0)

        if prediction.est >= threshold:  # Recommended
            recommended_items.append(1)
        else:  # Not recommended
            recommended_items.append(0)

    # Calculate Recall and Precision
    recall = recall_score(true_positives, recommended_items)
    precision = precision_score(true_positives, recommended_items)

    recall_scores.append(recall)
    precision_scores.append(precision)

    # Sort predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x.est, reverse=True)

    # To Calculate Mean Reciprocal Rank (MRR)
    mrr = 0
    for i, prediction in enumerate(predictions):
        if prediction.r_ui >= threshold:
            mrr = 1 / (i + 1)  # Found the first relevant item
            break

    mrr_scores.append(mrr)

    # For Calculating MAP at k
    k = 5
    map_at_k = 0
    num_relevant = 0
    for i, prediction in enumerate(predictions[:k]):
        if prediction.r_ui >= threshold:
            num_relevant += 1
            map_at_k += num_relevant / (i + 1)

    if num_relevant > 0:
        map_at_k /= num_relevant

    map_at_k_scores.append(map_at_k)

    # To Calculate Normalized Discounted Cumulative Gain at k
    relevance_scores = [1 if r_ui >= threshold else 0 for r_ui in ground_truth]
    predicted_scores = [-score for score in predicted_ratings]  # Negate score

    # Calculate NDCG at k
    dcg = 0
    idcg = 0
    for i in range(k):
        if i < len(predictions):
            dcg += (2 ** relevance_scores[i] - 1) / np.log2(i + 2)
        idcg += (2 ** relevance_scores[i] - 1) / np.log2(i + 2)
    if idcg > 0:
        ndcg = dcg / idcg
    else:
        ndcg = 0

    ndcg_scores.append(ndcg)


avg_rmse = np.mean(rmse_scores)
avg_recall = np.mean(recall_scores)
avg_precision = np.mean(precision_scores)
avg_mrr = np.mean(mrr_scores)
avg_map_at_k = np.mean(map_at_k_scores)
avg_ndcg = np.mean(ndcg_scores)


print(f"Average RMSE: {avg_rmse}")
print(f"Average Recall: {avg_recall}")
print(f"Average Precision: {avg_precision}")
print(f"Average MRR: {avg_mrr}")
print(f"Average MAP at {k}: {avg_map_at_k}")
print(f"Average NDCG at {k}: {avg_ndcg}")



Average RMSE: 255.3506801097219
Average Recall: 0.9999620829140269
Average Precision: 0.48684934480215936
Average MRR: 0.8555555555555556
Average MAP at 5: 0.8500000000000001
Average NDCG at 5: 1.0


In [ ]:
#recommendations
def recommend_items(customer_id, date, num_recommendations=5):

    filter = df[(df['CustomerID'] == customer_id) & (df['InvoiceDate'] < date)]
    not_purchased = df[~df['Description'].isin(filter['Description'])]['Description'].unique()
    recommendations = [model.predict(customer_id, item) for item in not_purchased]
    recommendations = sorted(recommendations, key=lambda x: x.est, reverse=True)
    top_recommendations = recommendations[:num_recommendations]
    recommended_descriptions = [rec.iid for rec in top_recommendations]
    return recommended_descriptions


customer_id = input("Enter Customer ID: ")
date_str = input("Enter Date (YYYY-MM-DD): ")


try:
    date = datetime.strptime(date_str, '%Y-%m-%d')
except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD format.")
    exit(1)

recommended_items = recommend_items(customer_id, date)
print(f"Recommended items for Customer {customer_id} on {date}: {recommended_items}")

Enter Customer ID: 14688
Enter Date (YYYY-MM-DD): 2012-01-10
Recommended items for Customer 14688 on 2012-01-10 00:00:00: ['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.']
